In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import joblib

# Load data
df = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")

# Convert TotalCharges to numeric
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")
df.dropna(subset=["TotalCharges"], inplace=True)

# Drop customerID
df.drop("customerID", axis=1, inplace=True)

# Target variable
y = df["Churn"].apply(lambda x: 1 if x == "Yes" else 0)
X = df.drop("Churn", axis=1)

# Feature columns
categorical_cols = X.select_dtypes(include=["object"]).columns.tolist()
numerical_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()

# Preprocessor
preprocessor = ColumnTransformer(transformers=[
    ("num", StandardScaler(), numerical_cols),
    ("cat", OneHotEncoder(drop="first", handle_unknown="ignore"), categorical_cols)
])

# Split data
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

# Pipelines
lr_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", LogisticRegression(random_state=42, max_iter=1000))
])

dt_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", DecisionTreeClassifier(random_state=42))
])

# Train models
lr_pipeline.fit(X_train, y_train)
dt_pipeline.fit(X_train, y_train)

# Predict
y_pred_lr = lr_pipeline.predict(X_val)
y_prob_lr = lr_pipeline.predict_proba(X_val)[:, 1]

y_pred_dt = dt_pipeline.predict(X_val)
y_prob_dt = dt_pipeline.predict_proba(X_val)[:, 1]

# Evaluate function
def evaluate(name, y_true, y_pred, y_prob):
    print(f"\n{name} Performance:")
    print(f"  Accuracy : {accuracy_score(y_true, y_pred):.4f}")
    print(f"  Precision: {precision_score(y_true, y_pred):.4f}")
    print(f"  Recall   : {recall_score(y_true, y_pred):.4f}")
    print(f"  F1-score : {f1_score(y_true, y_pred):.4f}")
    print(f"  ROC AUC  : {roc_auc_score(y_true, y_prob):.4f}")

evaluate("Logistic Regression", y_val, y_pred_lr, y_prob_lr)
evaluate("Decision Tree", y_val, y_pred_dt, y_prob_dt)

# Choose best model based on F1-score
f1_lr = f1_score(y_val, y_pred_lr)
f1_dt = f1_score(y_val, y_pred_dt)

best_model = lr_pipeline if f1_lr >= f1_dt else dt_pipeline

# Final evaluation on test set
y_test_pred = best_model.predict(X_test)
print("\nFinal Test Set Performance:")
print(f"Accuracy : {accuracy_score(y_test, y_test_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_test_pred):.4f}")
print(f"Recall   : {recall_score(y_test, y_test_pred):.4f}")
print(f"F1-score : {f1_score(y_test, y_test_pred):.4f}")

# Save the best model and preprocessor
joblib.dump(best_model, "model.pkl")
joblib.dump(preprocessor, "preprocessor.pkl")
joblib.dump(best_model, "pipeline.pkl")

print("\n✅ Saved model.pkl and preprocessor.pkl")



Logistic Regression Performance:
  Accuracy : 0.8237
  Precision: 0.6939
  Recall   : 0.6050
  F1-score : 0.6464
  ROC AUC  : 0.8533

Decision Tree Performance:
  Accuracy : 0.7166
  Precision: 0.4702
  Recall   : 0.5053
  F1-score : 0.4871
  ROC AUC  : 0.6507

Final Test Set Performance:
Accuracy : 0.7867
Precision: 0.6141
Recall   : 0.5286
F1-score : 0.5681

✅ Saved model.pkl and preprocessor.pkl


In [5]:
pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 830.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.5 MB/s eta 0:00:00


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501



⠙⠹⠸
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.80.9.43:8501

⠼⠴⠦⠧your url is: https://ready-states-give.loca.lt


In [10]:
print(preprocessor.feature_names_in_)


['gender' 'SeniorCitizen' 'Partner' 'Dependents' 'tenure' 'PhoneService'
 'MultipleLines' 'InternetService' 'OnlineSecurity' 'OnlineBackup'
 'DeviceProtection' 'TechSupport' 'StreamingTV' 'StreamingMovies'
 'Contract' 'PaperlessBilling' 'PaymentMethod' 'MonthlyCharges'
 'TotalCharges']


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7032 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7032 non-null   object 
 1   SeniorCitizen     7032 non-null   int64  
 2   Partner           7032 non-null   object 
 3   Dependents        7032 non-null   object 
 4   tenure            7032 non-null   int64  
 5   PhoneService      7032 non-null   object 
 6   MultipleLines     7032 non-null   object 
 7   InternetService   7032 non-null   object 
 8   OnlineSecurity    7032 non-null   object 
 9   OnlineBackup      7032 non-null   object 
 10  DeviceProtection  7032 non-null   object 
 11  TechSupport       7032 non-null   object 
 12  StreamingTV       7032 non-null   object 
 13  StreamingMovies   7032 non-null   object 
 14  Contract          7032 non-null   object 
 15  PaperlessBilling  7032 non-null   object 
 16  PaymentMethod     7032 non-null   object 
 17  